In [1]:
from lagom.envs import make_vec_env
from lagom.envs import wrap_atari
import gym
from agent import Agent


env = make_vec_env(lambda: wrap_atari(gym.make('PongNoFrameskip-v4')), 1, 0)
agent = Agent({'agent.replay_size': 10, 'agent.eps_train': 10, 'agent.eps_decay_period': 1, 
               'agent.min_replay_history': 100, 'agent.lr': 1e-3, 'agent.batch_size': 32,
              'agent.gamma': 0.99, 'agent.max_grad_norm': 40}, env, 'cpu')

/home/zuo/Code/tmp/lagom/lagom/vis/__init__.py:11: UserWarning: ImageViewer failed to import due to pyglet. 
  warnings.warn('ImageViewer failed to import due to pyglet. ')
/home/zuo/Code/tmp/lagom/lagom/envs/vec_env.py:12: UserWarning: ImageViewer failed to import due to pyglet. 
  warnings.warn('ImageViewer failed to import due to pyglet. ')


In [2]:
import torch
import numpy as np
agent.q_net(torch.from_numpy(np.asarray(env.reset())).)

tensor([[-0.0166, -0.0166, -0.0384,  0.0003,  0.0302,  0.0060]],
       grad_fn=<AddmmBackward>)

In [10]:
env.reset()[0].shape

(84, 84, 4)

In [9]:
a = torch.randn(3, 2)
print(a)
print(a.argmax(-1))
print(a.max(-1)[1])

tensor([[ 1.2632,  0.9043],
        [ 0.8153,  0.5093],
        [-0.3550,  0.3992]])
tensor([0, 0, 1])
tensor([0, 0, 1])


In [78]:
from replay_buffer import ReplayBuffer
from lagom import RandomAgent

env = make_vec_env(lambda: wrap_atari(gym.make('PongNoFrameskip-v4')), 1, 0)

replay = ReplayBuffer(30)
agent = RandomAgent(None, env, 'cpu')

def initialize_replay(config, agent, env, replay):
    random_agent = RandomAgent(None, env, None)
    observation = env.reset()
    for t in range(config['agent.min_replay_history']):
        action = agent.choose_action(observation)['raw_action']
        next_observation, reward, done, info = env.step(action)
        [replay.add(observation[i], action[i], reward[i], next_observation[i], done[i]) for i in range(len(env))]
        observation = next_observation
        if done:
            observation = env.reset()
            
initialize_replay({'agent.min_replay_history': 50}, agent, env, replay)


In [81]:
deque(maxlen=100)

deque([])

In [84]:
env.spec.max_episode_steps

400000

In [32]:
from lagom.envs import make_vec_env, wrap_atari
import gym
seed = 0
def make_env():
    return wrap_atari(gym.make('PongNoFrameskip-v4'))
env = make_vec_env(make_env, 1, seed, 'serial')


running_rewards = deque([0.0], maxlen=100)
env.reset()
for i in range(3000):
    obs, reward, done, info = env.step([env.action_space.sample()])
    running_rewards[-1] += reward[0]
    if done[0]:
        print(done)
        print(info[0]['terminal_observation'].shape)
        print(f'{i}: done')
        running_rewards.append(0.0)
    
print(running_rewards)


[True]
(84, 84, 4)
961: done
[True]
(84, 84, 4)
1779: done
[True]
(84, 84, 4)
2811: done
deque([-20.0, -21.0, -18.0, -4.0], maxlen=100)


In [40]:
env.spec.max_episode_steps

400000

In [58]:
gym.envs.atari.AtariEnv

<TimeLimit<AtariEnv<PongNoFrameskip-v4>>>

In [ ]:
a = [np.random.randn(84, 84, 4) for _ in range(100000)]
